In [13]:
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import numba
from numba import njit
from numba.typed import Dict
from numba.typed import List
import tensorly as tl
from tensorly.decomposition import parafac
import matplotlib.pyplot as plt

In [ ]:
@njit
def process_pair(fuck8, n, x, total_pairs, tensor, date_index):
    for index in range(total_pairs):
        i = x[index // n]
        j = x[index % n]
        x1 = x.index(i)
        y1 = x.index(j)
        fuck10_values = np.array(
            [fuck8[f"{i}r"], fuck8[f"{j}ryi"], fuck8[f"{j}rys"]])
        if (np.isnan(fuck10_values[0]) or np.isnan(fuck10_values[1]) or np.isnan(fuck10_values[2])):
            continue
        if (fuck10_values[0] >= 0):
            if fuck10_values[0] <= fuck10_values[1] and fuck10_values[0] <= fuck10_values[2]:
                tensor[x1, y1, date_index] = 1
        if (fuck10_values[0] < 0):
            if fuck10_values[0] <= fuck10_values[2] and fuck10_values[0] <= fuck10_values[1]:
                tensor[x1, y1, date_index] = 1
    return tensor

In [ ]:
fuck7 = pd.read_csv("finally.csv")
fuck7['datadate'] = pd.to_datetime(fuck7['datadate'])
fuck7 = fuck7.sort_values(by='datadate')
start_date = datetime.strptime("2013-03-15", "%Y-%m-%d")
end_date = datetime.strptime("2023-03-15", "%Y-%m-%d")
date_range = pd.date_range(start_date, end_date, freq='D')

with open("newstocks.txt", "r") as f:
    x = f.read().splitlines()
x[0] = "A"

n = len(x)
total_pairs = n * n
typed_x = List(x)
tensor = np.zeros((n, n, len(date_range)), dtype=np.int64)

for date_index, date in enumerate(date_range):
    typed_dict = Dict.empty(
        key_type=numba.types.unicode_type,
        value_type=numba.types.float64)
    fuck8 = fuck7[fuck7["datadate"] == date]

    if not fuck8.empty:
        fuck8 = fuck8.drop("datadate", axis=1)
        fuck8 = fuck8.drop("Unnamed: 0", axis=1)
        fuck8 = fuck8.to_dict()
        for key, value in fuck8.items():
            typed_dict[key] = value[3328]
        tensor = process_pair(typed_dict, n, typed_x, total_pairs, tensor, date_index)

In [ ]:
# Perform Tensor PCA
tl.set_backend('numpy') 
# parameter that determines the number of factors/components we want to
# extract from the tensor decomposition.
rank = 10
# Tensor PCA using PARAFAC (Parallel Factors Analysis) method
# decomposes the input tensor into a sum of rank-1 tensors
factors = parafac(tensor, rank)

# returns a list of factors, one for each mode (dimension) of the input tensor
# In our case, we have three modes: stock symbols (rows), stock symbols (columns), and dates (depth)
# Each factor is a 2D matrix where the number of rows corresponds to the size of the corresponding mode,
# and the number of columns is equal to the specified rank
for i, factor in enumerate(factors):
    print(f"Factor {i + 1}:")
    print(factor)
    print("\n")

In [ ]:
for i, factor in enumerate(factors):
    plt.figure(figsize=(12, 6))
    for j in range(rank):
        plt.plot(factor[:, j], label=f"Component {j + 1}")
    plt.title(f"Factor {i + 1}")
    plt.legend()
    plt.show()

# Calculate the explained variance ratio for each factor
explained_variances = []
total_variance = tl.norm(tensor, 2) ** 2

for component in range(rank):
    component_tensors = tl.kruskal_to_tensor((factors[0][:, component], factors[1][:, component], factors[2][:, component]))
    component_variance = tl.norm(component_tensors, 2) ** 2
    explained_variances.append(component_variance / total_variance)

for i, explained_variance in enumerate(explained_variances):
    print(f"Component {i + 1}: {explained_variance * 100:.2f}%")